In [1]:
import findspark
findspark.init()
import pyspark
import random
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
sc = SparkContext(appName='local') 
spark = SparkSession(sc)

<b>Explain your choice of processing framework briefly.</b>

Spark as compared to MapReduce is able to run more faster and effeciently than MapReduce both on memory and disk. This is due to
it's ability to perform work on the main worker node and prevent unnecessary I/O disk operations. Fcor iterative programs, it 
has a low iteration period the first time and is even lower in subsequent iterations compared to Hadoop.It even has a rider on 
it, in that it can easily output the data on disk if the memory is full.
As a programming language, it is also easy to set up and is rich in libraries/APIs making it easier to perform REPL operations


In [2]:
#Load 1987 data
basePath = "C:/Udvikling/edu/BigData/003/Data/"
airports = spark.read.format("csv").load(basePath + "airports.csv", header="true",inferSchema='true').show(10)
carriers = spark.read.format("csv").load(basePath + "carriers.csv", header="true",inferSchema='true').show(10)
flights = spark.read.format("csv").load(basePath + "flights/1987.csv", header="true",inferSchema='true')

+----+--------------------+----------------+-----+-------+-----------+------------+
|iata|             airport|            city|state|country|        lat|        long|
+----+--------------------+----------------+-----+-------+-----------+------------+
| 00M|            Thigpen |     Bay Springs|   MS|    USA|31.95376472|-89.23450472|
| 00R|Livingston Municipal|      Livingston|   TX|    USA|30.68586111|-95.01792778|
| 00V|         Meadow Lake|Colorado Springs|   CO|    USA|38.94574889|-104.5698933|
| 01G|        Perry-Warsaw|           Perry|   NY|    USA|42.74134667|-78.05208056|
| 01J|    Hilliard Airpark|        Hilliard|   FL|    USA| 30.6880125|-81.90594389|
| 01M|   Tishomingo County|         Belmont|   MS|    USA|34.49166667|-88.20111111|
| 02A|         Gragg-Wade |         Clanton|   AL|    USA|32.85048667|-86.61145333|
| 02C|             Capitol|      Brookfield|   WI|    USA|   43.08751|-88.17786917|
| 02G|   Columbiana County|  East Liverpool|   OH|    USA|40.67331278|-80.64

In [3]:
#How many flights were there from JFK to LAX?

flightsJFKLAX = flights.where("origin='JFK' and dest='LAX'").count()
print("Number of flights from JFK to LAX were {:,}".format(flightsJFKLAX))

Number of flights from JFK to LAX were 1,144


In [29]:
#What was the sum and average of all arrival delays for all delayed flights?


df1 = flights.select(sum("ArrDelay").alias ("x"))
df2 = flights.select(avg("ArrDelay").alias ("xy"))

df2.show()
#dfz = df1.withColumnRenamed("sum(ArrDelay)", "Sum").join(df2.withColumnRenamed("avg(ArrDelay)", "Average")).show()
dfz = df1.join(df2, ["xy"])

+-----------------+
|               xy|
+-----------------+
|9.446699049774669|
+-----------------+



AnalysisException: 'USING column `xy` cannot be resolved on the left side of the join. The left-side columns: [x];'

In [6]:
#What was the average departure delay for each state?

flights.groupBy("Origin").agg(round(avg("DepDelay"), 3).alias("Dep Delay")).sort("Dep Delay").show()

+------+---------+
|Origin|Dep Delay|
+------+---------+
|   ILG|   -2.407|
|   FOE|   -1.316|
|   LIH|     0.25|
|   EAU|    0.808|
|   PIE|    0.917|
|   APF|     1.02|
|   FAY|    1.094|
|   TRI|    1.591|
|   PFN|    1.594|
|   ILM|     1.74|
|   HSV|    1.828|
|   ATW|    1.925|
|   OAJ|    2.013|
|   CHA|    2.015|
|   CSG|    2.028|
|   AZO|    2.095|
|   SAV|    2.335|
|   PHF|    2.367|
|   SCC|    2.624|
|   VPS|    2.664|
+------+---------+
only showing top 20 rows



In [7]:
#Which airline performed the worst seen from a customer perspective? (This can be answered in many different ways. 

#Considering arrival delays as a decision parameter

flights.groupBy("UniqueCarrier").agg(avg("DepDelay").alias("Dep delays")).sort(desc("Dep delays")).show(5)
print("PS has the highest average delay at 13.319\n\n")

#Worst flights based on cancellations
cancelled = flights.select("UniqueCarrier").where("Cancelled==1").groupBy("UniqueCarrier").count().alias("cancelled")
total = flights.select("UniqueCarrier").groupBy("UniqueCarrier").count().alias("total")


dfx = cancelled.withColumnRenamed("count", "Cancelled").join(total.withColumnRenamed("count", "Total"), "UniqueCarrier")
dfx = dfx.withColumn("Pct% cancellations", expr("round((Cancelled/Total)*100, 2)")).sort(desc("Pct% cancellations"))
dfx.show(5)
 
print("EA had the highest percentage of cancellations and would be rated as the worst airline on this front.")

+-------------+------------------+
|UniqueCarrier|        Dep delays|
+-------------+------------------+
|           PS| 13.31959242658045|
|           CO|12.620661609706707|
|           EA| 8.832762739602341|
|           WN| 8.464397496087637|
|           AS| 8.302911613420791|
+-------------+------------------+
only showing top 5 rows

PS has the highest average delay at 13.319


+-------------+---------+------+------------------+
|UniqueCarrier|Cancelled| Total|Pct% cancellations|
+-------------+---------+------+------------------+
|           EA|     2454|108776|              2.26|
|           UA|     3436|152624|              2.25|
|           PS|      879| 41706|              2.11|
|           CO|     2508|123002|              2.04|
|           TW|     1243| 69650|              1.78|
+-------------+---------+------+------------------+
only showing top 5 rows

EA had the highest percentage of cancellations and would be rated as the worst airline on this front.


In [8]:
#Which airport performed the worst seen from a customer perspective? (This can also be answered in many different ways. 
#Describe what you consider.)

flights.groupBy("Origin").agg(avg("DepDelay").alias("Dep delays")).sort(desc("Dep delays")).show(5) 

+------+------------------+
|Origin|        Dep delays|
+------+------------------+
|   PIR|              85.0|
|   GUC|22.884615384615383|
|   ACV| 19.36873156342183|
|   MFR|17.509225092250922|
|   EUG|16.100617828773167|
+------+------------------+
only showing top 5 rows



In [9]:


flights.show(1)


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1987|   10|        14|        3|    741|       730|    912|       849|           PS|     1451